### Import the project from BigML

In [ ]:
#BigML project
from pandas import read_csv
import bigml.api
from bigml.api import BigML
api= BigML (project='project/5d94a52e5a213962e2000331')

### 1st Model (ensemble) on train splitted dataset

In [ ]:
#Upload the modified train set
source = api.create_source('fulltrain_dataframe.csv')

In [ ]:
api.ok(source)

In [ ]:
#Create the dataset
dataset_1 = api.create_dataset(source)

In [ ]:
api.ok(dataset_1)

In [ ]:
#Split the trainfull in train80/val20
train_dataset = api.create_dataset(
    dataset_1, {"name": "Train80",
                     "sample_rate": 0.8, "seed": "my seed"})
val = api.create_dataset(
    dataset_1, {"name": "Train20",
                     "sample_rate": 0.8, "seed": "my seed",
                     "out_of_bag": True})

In [ ]:
api.ok(train_dataset)

In [ ]:
api.ok(val)

In [ ]:
api.download_dataset(train_dataset, filename='train_dataset.csv')

In [ ]:
api.download_dataset(val, filename='val.csv')

In [ ]:
#Launch the model ensemble
model = api.create_ensemble(train_dataset, {"objective_field":"SeriousDlqin2yrs"})

In [ ]:
api.ok(model)

In [ ]:
#Launch the batch prediction on the validation set
batch_prediction = api.create_batch_prediction(model, val, {
    "all_fields": True, "confidence": True, "probabilities": True})

In [ ]:
api.ok(batch_prediction)

In [ ]:
#Download the prediction 
api.download_batch_prediction(batch_prediction,
    filename='../handson-ml2/prediction_credit.csv')

### Apply the Model on the test set

In [ ]:
#Upload the modified test
test_source = api.create_source('test_dataframe.csv')

In [ ]:
api.ok(test_source)

In [ ]:
#Create the dataset test
dataset_2 = api.create_dataset(test_source)

In [ ]:
api.ok(dataset_2)

In [ ]:
#Model ensemble on the test set
model = api.create_ensemble(dataset_1, {"objective_field":"SeriousDlqin2yrs"})

In [ ]:
api.ok(model)

In [ ]:
#Batch prediction on the test set + add column probabilities
batch_prediction = api.create_batch_prediction(model, dataset_2, {"all_fields": True, "confidence": True, "probabilities": True})

In [ ]:
api.ok(batch_prediction)

In [ ]:
#Evaluation on the test
evaluation = api.create_evaluation(model, dataset_1)
api.ok(evaluation)

In [ ]:
 api.pprint(evaluation['object']['result']['model']['average_area_under_roc_curve'])

In [ ]:
api.download_batch_prediction(batch_prediction,
    filename='../Credit_p/prediction_credit_final.csv')

In [ ]:
# Import the new prediction and remove the columns/replace Id and Probability
import pandas as pd
df = pd.read_csv('prediction_credit_final.csv')
df.drop(df.columns[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]], axis=1, inplace=True)
df.rename(columns={'field1': 'Id', '1 probability': 'Probability'}, inplace=True)
df.to_csv('prediction_credit_final_clean.csv', index=False)

In [ ]:
import kaggle
kaggle.api.competition_submit("prediction_credit_final_clean.csv", "test", "House Prices: Advanced Regression Techniques")

In [1]:
score = 0.85753
print("Kaggle score :", score)

Kaggle score : 0.85753
